### Modelo de Flujo de Caja (Cash Flow)

Este notebook contiene dos modelos de analisis financiero que tienen como objeto de estudio los diferentes modelo de VE que se encuentran en el Dataset "ElectricCarData_Norm.csv". Estos modelos estudian el comportamiento financiero de cada prototipo del vehiculo materializado en el calculo de un Flujo de Caja (Cash Flow) proyectado a 5 años, e indicadores financiero como: ROI (Return on Investment), IR (Indice de Rentabilidad) y Payback Period.

El primer modelo recibe como parametro una instancia (modelo de VE) del dataset y devuelve como resultado el comportamiento de este en relacion a las variables financieras anteriormente expuestas y tambien el del un auto convencional para obtener un analisis comparativo y poder evaluar diferencias y el costo de oportunidad de estas dos variantes.


Importamos las librerias necesarias.

In [185]:
import pandas as pd 

Importamos el dataset que contiene las instancias y variables necesarias. (Este dataset contiene los valores futuros, producto del modelo de ML)

In [186]:
#future_data = pd.read_csv(r"Prediccion_Dataset.csv", index_col = 0)
future_data = pd.read_csv(r"..\datasets\2. Depurados\TLC Aggregated Data\ML_TS_Output_Anualized.csv")
future_data = future_data[future_data['industry']=='FHV - High Volume']

Visualizamos el Dataset.

In [187]:
future_data.head(5)

,industry,year,anual_income_per_vehicle,anual_distance_per_vehicle,anual_hours_per_driver,anual_total_co2_emission
0,FHV - High Volume,2021,61111.547221,11579.379920,2453.100299,269089.638654
1,FHV - High Volume,2022,64772.609515,11981.139672,2354.199683,341462.060682
2,FHV - High Volume,2023,66497.771259,12108.181763,2329.998311,378616.000425
3,FHV - High Volume,2024,65612.343006,11771.171315,2230.171288,390179.064210
4,FHV - High Volume,2025,65357.057024,11762.559488,1939.063082,390936.358803


In [188]:
future_data.rename(columns={
    'anual_income_per_vehicle': 'Income_per_Vehicle (USD)',
    'anual_distance_per_vehicle': 'Miles_per_Vehicle',
    'anual_hours_per_driver': 'Hours_per_Driver',
    'anual_total_co2_emissio': 'Total_CO2_Emissions'
}, inplace=True)
anual_hours_per_driver = future_data[future_data['year']<2025]['Hours_per_Driver'].mean()
future_data =  future_data[future_data['year']>=2025]

Utilizamos como instrumento de ponderacion la media o promedio para tener un primer acercamiento y entendimiento a las variables (ingreso anual y millas recorridas al año). 

In [189]:
promedio_ingresos = round(future_data["Income_per_Vehicle (USD)"].mean(),2)
promedio_millas = round(future_data["Miles_per_Vehicle"].mean(),2)
print(f'El promedio de ingresos anuales es: {promedio_ingresos}')
print(f'El promedio de millas recorridas anualmente es : {promedio_millas}')
print(f'El promedio de cantidad de horas de manejo : {anual_hours_per_driver}')

El promedio de ingresos anuales es: 64271.22
El promedio de millas recorridas anualmente es : 11653.4
El promedio de cantidad de horas de manejo : 2341.867394973464


Se asigna una variable a cada dato.

In [190]:
año = 2025
for _, row in future_data.iterrows():  # iterrows() en lugar de interrows()
    globals()[f'Ingresos_año_{año}'] = round(row["Income_per_Vehicle (USD)"],2)
    print(f'Los ingresos para el año {año} son {globals()[f"Ingresos_año_{año}"]}')
    año += 1

Los ingresos para el año 2025 son 65357.06
Los ingresos para el año 2026 son 64802.76
Los ingresos para el año 2027 son 64302.32
Los ingresos para el año 2028 son 63485.52
Los ingresos para el año 2029 son 63408.43


In [191]:
ingresos_años = []
for _, row in future_data.iterrows():
    ingresos_años.append(round(row["Income_per_Vehicle (USD)"],2))
print(ingresos_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43]


In [192]:
año = 2025
for _, row in future_data.iterrows():
    globals()[f'Millas_Año_{año}'] = round(row["Miles_per_Vehicle"],2)
    print(f'Las Millas recorridas en el año {año} son {globals()[f"Millas_Año_{año}"]}')
    año+=1

Las Millas recorridas en el año 2025 son 11762.56
Las Millas recorridas en el año 2026 son 11703.94
Las Millas recorridas en el año 2027 son 11656.52
Las Millas recorridas en el año 2028 son 11556.92
Las Millas recorridas en el año 2029 son 11587.08


In [193]:
millas_años = []
for _, row in future_data.iterrows():
    millas_años.append(round(row["Miles_per_Vehicle"],2))
print(millas_años)

[11762.56, 11703.94, 11656.52, 11556.92, 11587.08]


In [194]:
print(ingresos_años)
print(millas_años)

[65357.06, 64802.76, 64302.32, 63485.52, 63408.43]
[11762.56, 11703.94, 11656.52, 11556.92, 11587.08]


En esta sección se elabora el mecanismo para el cálculo del modelo de Flujo de Caja (Cash Flow) y los demas indicadores financieros.

A continuación se presentaran las siguientes características:

- Debido a que son dos tipos de autos con caracteristicas distintas se crean 2 clases para cada variante, que heredan de la clase madre «Auto».
- Las caracteristicas que estructuran la clase «Auto Convencional» corresponden a una ponderacion que representa un propotipo de auto convencional promedio
- Se crean funciones para el calculo de las variables financieras anteriormente mencionadas.
- El objeto de análisis es un modelo de VE que compone el dataset y tambien de un prototipo promedio de un auto convencional.
- El resultado es el dictámen financiero compuesto por las variables financieras expuestas.
- Obtención de análisis comparativo entre un VE y un auto convencional.
- Análisis Financiero de la totalidad de VE y ranking de los prototipos más rentables.

### Modelo de Flujo de Caja (Cash Flow) #1
Análisis Financiero personalizado (por modelo VE)

In [195]:
# Se crea la clase Auto y subclases para AutoConvencional y AutoElectrico
class Auto:
    def __init__(self, precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, ingresos_anuales, tasa_descuento):
        self.precio = precio
        self.iva = iva
        self.licencia_1 = licencia_1
        self.licencia_2 = licencia_2
        self.placa = placa
        self.inspeccion = inspeccion
        self.seguro = seguro
        self.recorrido_anual = recorrido_anual
        self.ingresos_anuales = ingresos_anuales
        self.tasa_descuento = tasa_descuento
    
    def inversion_inicial(self):
        raise NotImplementedError
    
    def costos_operativos(self):
        raise NotImplementedError
    
    def flujo_neto_anual(self):
        return self.ingresos_anuales - self.costos_operativos()
    
    def flujo_caja_proyectado(self, años=5):
        # Inversión inicial negativa porqué es el año 0
        inversion_inicial = -self.inversion_inicial()
        
        ingresos_año = [0]
        costos_año = [0]
        flujo_neto = [0]  # Año 0: 
        flujo_descuento = [inversion_inicial]  # Año 0 Inversión inicial negativa
        ingresos_descuento_año = [0]
        # Se calcula el flujo neto anual y mensual para cada año
        for año in range(1, años + 1):
            self.ingresos_anuales = ingresos_años[año-1] # Codigo nuevo
            self.recorrido_anual = millas_años[año-1] # Codigo nuevo
            flujo_neto_actual = self.flujo_neto_anual()
            flujo_neto.append(flujo_neto_actual)
            flujo_descuento_actual = flujo_neto_actual / (1 + self.tasa_descuento) ** año
            flujo_descuento.append(flujo_descuento_actual)
            ingresos_año.append(self.ingresos_anuales)
            ingresos_descuento_año.append(ingresos_años[año-1]  / (1 + self.tasa_descuento) ** año)
            costos_año.append(self.costos_operativos())
        
        # Se crea el DataFrame de resultados
        df_flujo_caja = pd.DataFrame({
            'Año': range(0, años + 1),
            'Ingresos': ingresos_año, 
            'Costos Operativos': costos_año,
            'Flujo Neto': flujo_neto,
            'Flujo Neto Descontado (10%)': flujo_descuento,
            'Ingresos Descontado (10%)': ingresos_descuento_año
        })
        # Calculo el flujo total descontado para el ROI
        return df_flujo_caja

    def calcular_roi(self, flujo_total):
        roi = ((1+ flujo_total/ self.inversion_inicial())**(1/5)-1)*100 #- self.inversion_inicial()
        return round(roi, 2)

    def calcular_ir(self, flujo_total):
        ir = flujo_total / abs(self.inversion_inicial())
        return round(ir, 2)

    def calcular_payback_period(self):
        flujo_acumulado = 0
        payback = 0
        for año in range(1, 5):
            flujo_anual = self.flujo_neto_anual()
            flujo_acumulado += flujo_anual
            if flujo_acumulado >= abs(self.inversion_inicial()):
                payback = año
                break
        return payback if payback > 0 else "No recuperado en 5 años"

class AutoConvencional(Auto): #Licencia_1 = unico pago, licencia_2 = anual
    def __init__(self, precio=36200, iva=0.08875, licencia_1=550,licencia_2=100, placa = 300,
                 seguro=5000, inspeccion = 150,
                 eficiencia=0.06, mantenimiento=0.15, recorrido_anual=promedio_millas, #mantenimiento = USD x Milla
                 ingresos_anuales=promedio_ingresos, tasa_descuento=0.10):
        super().__init__(precio, iva, licencia_1, licencia_2, placa, inspeccion,
                         seguro, recorrido_anual, ingresos_anuales, tasa_descuento) # Se usa el super constructor para evitar redundancia ya que es una clase hija
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
    
    def inversion_inicial(self):
        return self.precio * (1 + self.iva) + self.licencia_1
    
    def costos_operativos(self):
        precio_gasolina_litro = 0.92 #USD x lt
        costo_gasolina_anual = (self.recorrido_anual * self.eficiencia) * precio_gasolina_litro
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_salario = 2342 * 21  # Precio medio de la hora de un conductor de autos convencionales.
        return costo_gasolina_anual + costo_mantenimiento + self.seguro + self.licencia_2 + self.placa + self.inspeccion +costo_anual_salario

class AutoElectrico(Auto): #Descuento de 5% al 10% en licencia ; Excencion del impuesto de venta
    def __init__(self, precio, eficiencia, iva=0, licencia_1=495,licencia_2=100, placa = 300,
                 seguro=1500, inspeccion = 100, mantenimiento=0.03, wallbox=600, 
                 recambio_bateria=5000, #mantenimiento = USD x Milla
                 recorrido_anual=promedio_millas, ingresos_anuales=promedio_ingresos, tasa_descuento=0.10): #16529 #93227
        super().__init__(precio, iva, licencia_1, licencia_2, placa, inspeccion, seguro, recorrido_anual, ingresos_anuales, tasa_descuento)
        self.eficiencia = eficiencia
        self.mantenimiento = mantenimiento
        self.wallbox = wallbox
        self.recambio_bateria = recambio_bateria
    
    def inversion_inicial(self):
        return self.precio + self.licencia_1 + self.wallbox
    
    def costos_operativos(self):
        precio_electricidad_kWh = 0.13 #USD x kHw
        costo_electricidad_anual = (self.recorrido_anual * self.eficiencia) * precio_electricidad_kWh
        costo_mantenimiento = self.mantenimiento * self.recorrido_anual
        costo_anual_bateria = self.recambio_bateria / 12.5
        costo_anual_salario = 2342 * 22.5 # Promedio de horas de menejo de un conductor por año * Precio promedio de la hora de un chofer de taxis en NY.
        return costo_electricidad_anual + costo_mantenimiento + costo_anual_bateria + self.seguro + self.licencia_2 + self.placa + self.inspeccion + costo_anual_salario

# Función para seleccionar el modelo del auto, calcular flujos e indicadores
def comparar_flujo_caja_y_medir_indicadores(df_autos, modelo, tipo_auto):
    # Se Filtra en el dataset para obtener el precio del modelo
    filtro_modelo = df_autos[df_autos['Model'] == modelo]
    
    if filtro_modelo.empty:
        print(f"Modelo '{modelo}' no encontrado en el dataset.")
        return None
    
    precio = filtro_modelo['Precio Dolar'].values[0]
    eficiencia = filtro_modelo['Efficiency (kWh/mile)'].values[0]
    
    # Instancio el auto eléctrico y el auto convencional
    auto_electrico = AutoElectrico(precio, eficiencia)
    auto_convencional = AutoConvencional()
    
    # Se calcula el flujo de caja proyectado
    flujo_electrico = auto_electrico.flujo_caja_proyectado()
    flujo_convencional = auto_convencional.flujo_caja_proyectado()
    
    # Sumo flujo neto descontado para cada tipo de auto (obtener total)
    flujo_descuento_total_electrico = round(flujo_electrico['Flujo Neto Descontado (10%)'].sum(),2)
    flujo_descuento_total_convencional = round(flujo_convencional['Flujo Neto Descontado (10%)'].sum(),2)

    # Sumo ingresos descontado para cada tipo de auto (obtener total)
    ingresos_descuento_total_electrico = round(flujo_electrico['Ingresos Descontado (10%)'].sum(),2)
    ingresos_descuento_total_convencional = round(flujo_convencional['Ingresos Descontado (10%)'].sum(),2)

    # Calculo del ROI, IR y Payback Period para cada auto
    roi_electrico = auto_electrico.calcular_roi(flujo_descuento_total_electrico)
    roi_convencional = auto_convencional.calcular_roi(flujo_descuento_total_convencional)
    ir_electrico = auto_electrico.calcular_ir(ingresos_descuento_total_electrico)
    ir_convencional = auto_convencional.calcular_ir(ingresos_descuento_total_convencional)
    payback_electrico = auto_electrico.calcular_payback_period()
    payback_convencional = auto_convencional.calcular_payback_period()
    
    df_flujo_electrico = pd.DataFrame(flujo_electrico)
    df_flujo_convencional = pd.DataFrame(flujo_convencional)

    # Resultados
    print(f"Flujo de Caja Proyectado para {modelo} (Eléctrico):")
    print(df_flujo_electrico)
    
    print("\nFlujo de Caja Proyectado para Auto Convencional:")
    print(df_flujo_convencional)
    
    print("\nComparación Financiera:")
    print(f"Auto Eléctrico - Flujo Neto Descontado Total: {flujo_descuento_total_electrico}, ROI: {roi_electrico}%, IR: {ir_electrico} USD, Payback Period: {payback_electrico} años")
    print(f"Auto Convencional - Flujo Neto Descontado Total: {flujo_descuento_total_convencional}, ROI: {roi_convencional}%, IR: {ir_convencional} USD, Payback Period: {payback_convencional} años")

Ejemplo de uso:

In [196]:
# Dataset de VE
df_autos = pd.read_csv(r'..\datasets\2. Depurados\ElectricCarData_Clean.csv')#"df_modelo.csv")
df_autos.rename(columns={
    'model': 'Model',
    'efficiency_whkm': 'Efficiency (Wh/km)'
}, inplace=True)

df_autos['Efficiency (kWh/mile)'] = df_autos['Efficiency (Wh/km)']/1000* 0.1688666667 
df_autos['Precio Dolar'] = df_autos['price_euro'] * 1.06

In [197]:
modelo_auto = 'Mii Electric '  # El modelo elegido
comparar_flujo_caja_y_medir_indicadores(df_autos,modelo_auto, AutoElectrico)

Flujo de Caja Proyectado para Mii Electric  (Eléctrico):
   Año  Ingresos  Costos Operativos   Flujo Neto  Flujo Neto Descontado (10%)  \
0    0      0.00           0.000000     0.000000                -22431.740000   
1    1  65357.06       55490.741247  9866.318753                  8969.380685   
2    2  64802.76       55488.769027  9313.990973                  7697.513201   
3    3  64302.32       55487.173622  8815.146378                  6622.949946   
4    4  63485.52       55483.822665  8001.697335                  5465.266945   
5    5  63408.43       55484.837373  7923.592627                  4919.927618   

   Ingresos Descontado (10%)  
0                   0.000000  
1               59415.509091  
2               53556.000000  
3               48311.284748  
4               43361.464381  
5               39371.646249  

Flujo de Caja Proyectado para Auto Convencional:
   Año  Ingresos  Costos Operativos   Flujo Neto  Flujo Neto Descontado (10%)  \
0    0      0.00           

A continuacion se exponen los modelos contenidos en el dataset para agregarlos en le input de la funcion.

In [198]:
#df_autos["Model"].unique()

### Modelo de Flujo de Caja (Cash Flow) #2:
Ranking de rentabilidad financiera de todos los prototipos de VE contenidos en el dataset

In [199]:
# Definición de la función para calcular y comparar métricas
def calcular_metricas_para_todos_los_modelos(df_autos):
    resultados = []
    resultados_conv = []

    # Se itera sobre cada modelo en el dataset
    for _, row in df_autos.iterrows(): #uso _, ya que no necesito el indice
        modelo = row['Model']
        precio = row['Precio Dolar']
        eficiencia = row['Efficiency (kWh/mile)']
        # Se instancia AutoEléctrico y AutoConvencional con el precio del modelo actual
        auto_electrico = AutoElectrico(precio, eficiencia)
        auto_convencional = AutoConvencional()

        # Se calcula los flujos de caja proyectados para ambos tipos de autos
        flujo_electrico = auto_electrico.flujo_caja_proyectado()
        flujo_convencional = auto_convencional.flujo_caja_proyectado()

        # Luego, se calcula el flujo neto descontado total
        flujo_descuento_total_electrico = flujo_electrico['Flujo Neto Descontado (10%)'].sum()
        flujo_descuento_total_convencional = flujo_convencional['Flujo Neto Descontado (10%)'].sum()

        # Sumo ingresos descontado para cada tipo de auto (obtener total)
        ingresos_descuento_total_electrico = round(flujo_electrico['Ingresos Descontado (10%)'].sum(),2)
        ingresos_descuento_total_convencional = round(flujo_convencional['Ingresos Descontado (10%)'].sum(),2)


        # Calculo ROI, IR y Payback Period
        roi_electrico = auto_electrico.calcular_roi(flujo_descuento_total_electrico)
        ir_electrico = auto_electrico.calcular_ir(ingresos_descuento_total_electrico)
        payback_electrico = auto_electrico.calcular_payback_period()

                # Calculo ROI, IR y Payback Period
        roi_convencional = auto_convencional.calcular_roi(flujo_descuento_total_convencional)
        ir_convencional = auto_convencional.calcular_ir(ingresos_descuento_total_convencional)
        payback_convencional = auto_convencional.calcular_payback_period()

        # Se agg los resultados a la lista
        resultados.append({
            'Modelo': modelo,
            'Precio (USD)': round(precio,1),
            'Flujo Neto Descontado (USD)': round(flujo_descuento_total_electrico,1),
            'ROI (%)': roi_electrico,
            'IR (USD)': ir_electrico,
            'Payback Period (Años)': payback_electrico
        })

        # Se agg los resultados a la lista
        resultados_conv.append({
            'Modelo': "Convenncional",
            'Precio (USD)': 15000,
            'Flujo Neto Descontado (USD)': round(flujo_descuento_total_convencional,1),
            'ROI (%)': roi_convencional,
            'IR (USD)': ir_convencional,
            'Payback Period (Años)': payback_convencional
        })

        # Se convierte a DataFrame
    df_resultados = pd.DataFrame(resultados)
    df_resultados_conv = pd.DataFrame(resultados_conv)
    # Se obtienen los 5 mejores con nlargest
    # top_5_modelos = df_resultados.nlargest(5, ['ROI (%)', 'IR (USD)', 'Flujo Neto Descontado (USD)'])
    df_resultados = df_resultados.sort_values(by="ROI (%)", ascending=False)
    df_resultados_conv = df_resultados_conv.sort_values(by="ROI (%)", ascending=False)

    return df_resultados, df_resultados_conv
    #return top_5_modelos

# Se invoca la función para calcular métricas y obtener los top 5 modelos
df_resultados,df_resultados_conv = calcular_metricas_para_todos_los_modelos(df_autos)

df_resultados.head(5)

,Modelo,Precio (USD),Flujo Neto Descontado (USD),ROI (%),IR (USD),Payback Period (Años)
44,Mii Electric,21336.7,11243.3,8.46,10.88,3
82,EQ fortwo coupe,22670.2,9908.8,7.22,10.27,3
17,e-Up!,22706.3,9873.8,7.19,10.25,4
77,EQ forfour,23351.8,9218.5,6.61,9.98,4
43,CITIGOe iV,26006.0,6574.0,4.44,9.00,4


In [200]:
df_resultados_conv.head(5)

,Modelo,Precio (USD),Flujo Neto Descontado (USD),ROI (%),IR (USD),Payback Period (Años)
0,Convenncional,15000,-12496.5,-7.23,6.11,No recuperado en 5 años
1,Convenncional,15000,-12496.5,-7.23,6.11,No recuperado en 5 años
2,Convenncional,15000,-12496.5,-7.23,6.11,No recuperado en 5 años
3,Convenncional,15000,-12496.5,-7.23,6.11,No recuperado en 5 años
4,Convenncional,15000,-12496.5,-7.23,6.11,No recuperado en 5 años
